In [1]:
import tensorflow as tf

# 0 、文件位置

In [2]:
trainset_path = "./image/train"
testset_path = "./image/test"
output_path = "./output"

image_width = 24
image_height = 24
image_channels = 3

#总共3类
num_breed = 3

# 一、创建 TFRecords文件

In [3]:
import glob
import os
from itertools import groupby
from scipy import misc
import matplotlib.pylab as pl
import matplotlib.pyplot as plt

In [4]:
# class 信息
breeds_with_path = glob.glob(trainset_path + "/*")
breeds = list(map(lambda x:x.split("/")[3],breeds_with_path))
breeds_dict = dict(zip(breeds, [x for x in range(len(breeds))]))
#print(breeds_with_path,breeds_dict,[x for x in range(len(breeds))])

In [5]:
def create_record(dataset_path,output_location):
    # 获取数据集下所有文件路径
    dataset_files = glob.glob(dataset_path + "/*/*.jpg")
    # 分离出claess信息,得到（class,file_path）对的列表
    image_filepath_with_breed = list(map(lambda file_path:(file_path.split("/")[3],file_path),dataset_files))
    
    #每个TFRecord文件保存10个图片
    nums_per_file = 10
    writer = None
    
    # 记录当前读到的图像index
    current_index = 0
    for breed, file_path in image_filepath_with_breed:
        if current_index % nums_per_file == 0:
            if writer:
                print("current_index = ",current_index,"\n")
                writer.close()
            
            # 格式化字符串
            record_filename = "{output_location}{current_index}.tfrecords".format(
            output_location = output_location,
            current_index = current_index)
            
            writer = tf.python_io.TFRecordWriter(record_filename)
            
        current_index += 1
        
        # 读取图像并转换为tf.example格式
        image = misc.imread(file_path)
            
        # 图像转换为Byte型
        image_raw = image.tobytes()
        
        example = tf.train.Example(features = 
                                  tf.train.Features(feature = {
                    "label": tf.train.Feature(int64_list = tf.train.Int64List(value = [breeds_dict[breed]])),
                    "image_raw": tf.train.Feature(bytes_list = tf.train.BytesList(value = [image_raw]))   
                }     
            )    
        )
        
        # 写入文件
        writer.write(example.SerializeToString())
        
    # 关闭最后一个文件
    writer.close()

In [6]:
#create_record(trainset_path,output_path + "/training-images/")
#create_record(testset_path,output_path + "/testing-images/")
print("finished\n")

finished



# 二、 读取TFRecord 文件

In [7]:
filenames =tf.train.match_filenames_once(output_path + "/training-images/*.tfrecords")
#["/home/lile/ML_learning/ML_framework/output/training-images/10.tfrecords","/home/lile/ML_learning/ML_framework/output/training-images/0.tfrecords"] #
filenames_queue = tf.train.string_input_producer(filenames, shuffle = True)

reader = tf.TFRecordReader()
_, serialized_example = reader.read(filenames_queue)
features = tf.parse_single_example(
    serialized_example,
    features = {
        "label":tf.FixedLenFeature([],tf.int64),
        "image_raw":tf.FixedLenFeature([],tf.string)
    }
)

# 解码图像数据
im = tf.decode_raw(features["image_raw"],tf.uint8) 
reshape = tf.reshape(im,(image_height,image_width,image_channels))
image = tf.cast(reshape,tf.float32)
label = tf.cast(features["label"],tf.int64)


# 组合训练数据
batch_size = 5
min_after_dequeue = 100*batch_size
capacity = min_after_dequeue + 3*batch_size

image_batch, label_batch = tf.train.shuffle_batch(
(image,label),batch_size = batch_size,
capacity = capacity,min_after_dequeue = min_after_dequeue
)

# 三、定义网络模型

In [8]:
###### 各层图片尺寸 
# input   [batch_size, 224, 224, 3]
# conv1   [32, 56, 56, 64]
# pool1   [32, 27, 27, 64]
# conv2   [32, 27, 27, 192]
# pool2   [32, 13, 13, 192]
# conv3   [32, 13, 13, 384]
# conv4   [32, 13, 13, 256]
# conv5   [32, 13, 13, 256]
# pool5   [32, 6, 6, 256]


# 定义w,b初始化函数
def variable_with_weight_loss(shape, stddev, wl):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), wl, name='weight_loss')
        tf.add_to_collection('losses', weight_loss)
    return var


#def interface(x):
    # 输入层
with tf.name_scope("input") as scope:
    x = tf.placeholder(tf.float32,shape = [None, image_height, image_width, image_channels], name = "x_input")
    y = tf.placeholder(tf.int64,shape = [batch_size],name = "y_input")
    #x = image_batch
    #y = label_batch
    tf.summary.image("image_input",x, batch_size)

    #卷积层一
with tf.name_scope("conv1") as scope:
    weight1 = variable_with_weight_loss(shape=[5, 5, 3, 64], stddev=5e-2, wl=0.0)
    kernel1 = tf.nn.conv2d(x, weight1, [1, 1, 1, 1], padding='SAME')
    bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
    conv1 = tf.nn.relu(tf.nn.bias_add(kernel1, bias1))
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    
    tf.summary.histogram("weight1",weight1)
    
    #卷积层二
with tf.name_scope("conv2") as scope:
    weight2 = variable_with_weight_loss(shape=[5, 5, 64, 64], stddev=5e-2, wl=0.0)
    kernel2 = tf.nn.conv2d(norm1, weight2, [1, 1, 1, 1], padding='SAME')
    bias2 = tf.Variable(tf.constant(0.1, shape=[64]))
    conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    tf.summary.histogram("weight2",weight2)

    #全连接层一
with tf.name_scope("full_connect1") as scope:
    reshape = tf.reshape(pool2, [batch_size, -1])
    weight3 = variable_with_weight_loss(shape=[6*6*64, 384], stddev=0.04, wl=0.004)
    bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
    local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)
    
    tf.summary.histogram("weight3",weight3)
    
    #全连接层二
with tf.name_scope("full_connect2") as scope:
    weight4 = variable_with_weight_loss(shape=[384, 192], stddev=0.04, wl=0.004)
    bias4 = tf.Variable(tf.constant(0.1, shape=[192]))                                      
    local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)
    
    tf.summary.histogram("weight4",weight4)
    
    #全连接层三
with tf.name_scope("full_connect3") as scope:
    weight5 = variable_with_weight_loss(shape=[192, 3], stddev=1/192.0, wl=0.0)
    bias5 = tf.Variable(tf.constant(0.0, shape=[3]))
    logits = tf.add(tf.matmul(local4, weight5), bias5)
    
    tf.summary.histogram("weight5",weight5)
    # softmax处理
    #y_ = tf.nn.softmax(fc3)
    
with tf.name_scope("cross_entropy") as scope:
    # 定义训练代价函数
    cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = y))

    tf.summary.scalar("cross_entropy",cross_entropy)
    
#cross_entropy = -tf.reduce_sum(y_*tf.log(y))
# 定义梯度优化算法
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# 评估精确度
#correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(y,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))

In [9]:
# 汇总数据
log_dir = "./logdir"
merged = tf.summary.merge_all()

#test_writer = tf.summary.FileWriter(log_dir + '/test')


# 运行

In [10]:
num_iter = 3000
init = [tf.global_variables_initializer(),tf.local_variables_initializer()]
with tf.Session() as sess:
    #在迭代控制中,记得添加tf.initialize_local_variables(),官网教程没有说明,但是如过不加，会出错
    sess.run(init)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess = sess, coord = coord)
    
    #y1,y2 = sess.run([image_batch, label_batch])
    
    train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
    #训练    
    print("start training......\n")
    for step in range(num_iter):
        
       
        images_batch,labels_batch = sess.run([image_batch,label_batch])
        #print(images_batch.shape,labels_batch.shape,images_batch.dtype,labels_batch.dtype)
        
        _ = sess.run(train_step, feed_dict = {x:images_batch,y:labels_batch})
        
        
        if step % 10 == 0:
            summary, loss_value  = sess.run([merged, cross_entropy],feed_dict = {x:images_batch,y:labels_batch})
            #format_str = ('step %d, loss = %.2f')
            print(step,loss_value)
            train_writer.add_summary(summary, step)
    
     
    train_writer.close()
    print("stop training.\n")
    
    coord.request_stop()
    coord.join(threads)
    
    print("finished.\n")
    

start training......

0 0.800055
10 1.13784
20 1.46381
30 1.04644
40 0.677248
50 1.20038
60 0.699789
70 1.21471
80 1.02347
90 1.17041
100 0.938611
110 0.712419
120 0.900339
130 0.96708
140 0.535829
150 1.03887
160 0.841516
170 1.64269
180 0.932998
190 1.14178
200 0.789508
210 1.15049
220 0.857313
230 1.13796
240 0.515376
250 0.7717
260 0.356558
270 0.86998
280 0.485585
290 0.731056
300 0.699039
310 0.532303
320 0.631616
330 0.266884
340 0.549602
350 0.650543
360 0.682331
370 0.606728
380 0.545595
390 0.443321
400 0.738283
410 0.379079
420 0.613683
430 0.45776
440 0.391887
450 0.305967
460 0.130578
470 0.512787
480 0.649019
490 0.467492
500 0.132391
510 0.914404
520 0.468524
530 0.482923
540 0.339596
550 0.365091
560 0.117273
570 0.438364
580 0.512281
590 0.18732
600 0.399005
610 0.181386
620 0.123407
630 0.143097
640 0.074688
650 0.12731
660 0.149209
670 0.069649
680 0.229643
690 0.0977665
700 0.103226
710 0.304361
720 0.21018
730 0.102874
740 0.0859557
750 0.124427
760 0.0983168
770 0